In [ ]:
cd /content/drive/My\ Drive/lane_follower

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

In [ ]:

import cv2
import time
import os
import pandas as pd
import numpy as np
from numpy import array
from tqdm import tqdm
from collections import deque
from random import shuffle
import tflearn
from google.colab.patches import cv2_imshow

In [11]:
from collections import Counter
from random import shuffle
HEIGHT = 100
WIDTH = 150
lr = 1e-3
td = []
image_1 = np.load('training_data/data_image.npy')
value_1 = np.load('training_data/data_value.npy')
image_2 = np.load('training_data/data_image_2.npy')
value_2 = np.load('training_data/data_value_2.npy')
image = np.concatenate((image_1, image_2), axis=0)
value = np.concatenate((value_1, value_2), axis=0)
print(image.shape)

(40002, 100, 150)


In [13]:
choice = []
for i in range(len(value)):
  if (value[i]==[2,0]).all():
    choice.append([1,0,0,0])
    #print(choice[i])
  elif (value[i]==[0,2]).all():
    choice.append([0,1,0,0])
    #print(choice[i])
  elif (value[i]==[0,-2]).all():
    choice.append([0,0,1,0])
    #print(choice[i])
  elif (value[i]==[0,0]).all():
    choice.append([0,0,0,1])
    #print(choice[i])
  else:
    print("error")

In [ ]:
for i in range(len(image)):
  td.append([image[i],choice[i]])
df = pd.DataFrame(td)
print(Counter(df[1].apply(str)))
Left = []
Right = []
Front = []
for data in td:
  image = data[0]
  value = data[1]
  #print(value)
  if (value == [1,0,0,0]):
    Front.append([image,value])
  elif (value == [0,0,1,0]):
    Right.append([image,value])
  elif (value == [0,1,0,0]):
    Left.append([image,value])
  else:
    pass

#print(len(Front))
Front = Front[:5000]
final_data = np.array(Front+Left+Right)
shuffle(final_data)
print(final_data.shape)

In [ ]:
train = final_data[:-500,:]
test=final_data[-500:,:]
print(train.shape)
X = np.array([i[0] for i in train]).reshape(-1,WIDTH,HEIGHT,1)
Y = [i[1] for i in train]
test_x = np.array([i[0] for i in test]).reshape(-1,WIDTH,HEIGHT,1)
test_y = [i[1] for i in test]
print(X[1].shape)

In [ ]:
from models import alexnet2
model = alexnet2(WIDTH,HEIGHT,lr,output=4)
model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
                snapshot_step=2500, show_metric=True)

In [19]:
images = np.load('training_data/data_image_2.npy')
value = np.load('training_data/data_value_2.npy')
choice = []
for i in range(len(value)):
  if (value[i]==[2,0]).all():
    choice.append([1,0,0,0])
    #print(choice[i])
  elif (value[i]==[0,2]).all():
    choice.append([0,1,0,0])
    #print(choice[i])
  elif (value[i]==[0,-2]).all():
    choice.append([0,0,1,0])
    #print(choice[i])
  elif (value[i]==[0,0]).all():
    choice.append([0,0,0,1])
    #print(choice[i])
  else:
    print("error")

In [ ]:
correct = 0
Left = []
Right = []
Front = []

for i in range(len(choice)):
  if (choice[i] == [1,0,0,0]):
    Front.append([images[i],choice[i]])
  elif (choice[i] == [0,0,1,0]):
    Right.append([images[i],choice[i]])
  elif (choice[i] == [0,1,0,0]):
    Left.append([images[i],choice[i]])
  else:
    pass


required = Left + Right
X = np.array([i[0] for i in required])
for i in range(1745):
  im = X[i].reshape(150,100,1)
  prediction = np.array(model.predict([im]))
  prediction = (prediction > 0.5)*1
  #print(prediction[0])
  if (prediction[0] == choice[i]).all():
    print("correct")
    correct+=1
  else:
    print("u r fucked")
print(correct)
print(X.shape)

In [ ]:
print(X.shape)
print(len(required))
model.save("final_2.model")